<a href="https://colab.research.google.com/github/Bystrooky-slepys/PIZ/blob/master/Semin%C3%A1rn%C3%AD_pr%C3%A1c_UI_%C5%A0J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generování datové sady

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split

def generate_shape(shape, size=(28,28)):
    img = Image.new('L', size, color=0)
    draw = ImageDraw.Draw(img)
    if shape == 'circle':
        draw.ellipse((4,4,24,24), outline=255, fill=255)
    elif shape == 'square':
        draw.rectangle((4,4,24,24), outline=255, fill=255)
    elif shape == 'triangle':
        draw.polygon([(14,4), (24,24), (4,24)], outline=255, fill=255)
    return np.array(img)

shapes = ['circle', 'square', 'triangle']
data = []
labels = []

# Generování 1000 obrázků pro každý tvar
for label, shape in enumerate(shapes):
    for _ in range(1000):
        img = generate_shape(shape)
        # Přidání náhodného šumu nebo transformace
        img = img + np.random.randint(0, 30, (28,28))
        img = np.clip(img, 0, 255)
        data.append(img)
        labels.append(label)

data = np.array(data)
labels = np.array(labels)

# Rozdělení na trénovací a testovací sadu
train_images, test_images, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels)


Funkce **generate_shape**: Vytváří obrázek daného tvaru.

Generování dat: Pro každý tvar vytvoříme 1000 obrázků s náhodnými variacemi.

Rozdělení dat: Data rozdělíme na trénovací (80 %) a testovací (20 %) sadu.

# Přepracování dat

In [ ]:
# Normalizace dat
train_images = train_images / 255.0
test_images = test_images / 255.0

# Přidání kanálové dimenze
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


Normalizace: Škálování pixelů na interval [0,1][0,1].

Reshape: Přizpůsobení tvaru dat pro CNN.

# Definice a kompilace modelu

In [ ]:
from tensorflow.keras import layers, models

# Definice modelu
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 třídy: kruh, čtverec, trojúhelník
])

# Kompilace modelu
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Architektura modelu: Lehčí model vzhledem k jednoduchosti dat.

Výstupní vrstva: Má 3 neurony odpovídající třem třídám.

# Trénování modelu

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTestovací přesnost: {test_acc:.2f}')


pochs: 10 epoch pro dostatečné naučení modelu.

Validation_split: 20 % trénovacích dat pro validaci během trénování.

# Vyhodnocení modelu

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTestovací přesnost: {test_acc:.2f}')


Výsledek: Získáme přesnost modelu na testovací sadě.

# Vizualizace výsledků

In [ ]:
# Graf trénovací a validační přesnosti
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Trénovací přesnost')
plt.plot(history.history['val_accuracy'], label='Validační přesnost')
plt.xlabel('Epoka')
plt.ylabel('Přesnost')
plt.legend()
plt.show()

# Zobrazení několika predikcí
import numpy as np

predictions = model.predict(test_images)
class_names = ['Circle', 'Square', 'Triangle']

def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i].reshape(28,28)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions_array[i])
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'
    plt.xlabel(f"Predikce: {class_names[predicted_label]}\nSkutečnost: {class_names[true_label]}", color=color)

num_rows = 3
num_cols = 3
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(9):
    plt.subplot(num_rows, num_cols, i+1)
    plot_image(i, predictions, test_labels, test_images)
plt.tight_layout()
plt.show()


Graf přesnosti: Ukazuje, jak se model učil během trénování.

Zobrazení predikcí: Vizualizujeme 9 testovacích obrázků s predikovanými a skutečnými třídami.

Testovací přesnost: Očekává se vysoká přesnost (blízko 100 %) vzhledem k jednoduchosti úlohy.

Analýza chyb: Pokud model udělá chyby, můžeme je identifikovat pomocí vizualizací.